In [1]:
import numpy as np
import csv
import pandas as pd
import os
import torch
import matplotlib.pyplot as plt
import camera_tools as ct
import matplotlib.pyplot as plt
from numpy.core.function_base import linspace
from FableAPI.fable_init import api

api.setup(blocking=True)
moduleids = api.discoverModules()
print("Module IDs: ", moduleids)
moduleID = moduleids[0]
print("Battery level:",api.getBattery(moduleID),"%")

#Calibrate the camera to detect green box, if you haven't done this calibration before
low_green, high_green = ct.colorpicker()
print(low_green)
print(high_green)
#Check whether the camera detects the green object properly
cam = ct.prepare_camera()
img = ct.capture_image(cam)
ct.show_camera(cam)

#Use this function to calculate Y (angular) position errors 
# Y (angular) positions
def readAngleFilesAndCollectErrors(fileName):
    angleDataFrame = pd.read_csv(fileName)
    desired_column = angleDataFrame.columns[0]
    angle_error_list = []
    for i in range(1,len(angleDataFrame)):
        current_data_element = angleDataFrame[desired_column][i]
        previous_data_element = angleDataFrame[desired_column][i-1]
        #TODO: Calculate the Y (angular) position error(say current_error) as the difference between 
        # the current Y (angular) position and previous Y (angular) position
        current_error = current_data_element - previous_data_element
        #print(current_data_element," ", previous_data_element, " ", current_error)
        angle_error_list = angle_error_list + [current_error]
    return angle_error_list

#Use this function to calculate errors between x and y coordinates
def readXYCoordsFilesAndCollectErrors(fileName):
    xyCoordsDataFrame = pd.read_csv(fileName)
    x_col_name = xyCoordsDataFrame.columns[0]
    y_col_name = xyCoordsDataFrame.columns[1]
    x_pos_error_list = []
    y_pos_error_list = []
    for i in range(1,len(xyCoordsDataFrame)):
        current_data_element_x = xyCoordsDataFrame[x_col_name][i]
        current_data_element_y = xyCoordsDataFrame[y_col_name][i]
        previous_data_element_x = xyCoordsDataFrame[x_col_name][i-1]
        previous_data_element_y = xyCoordsDataFrame[y_col_name][i-1]
        error_x_pos = current_data_element_x - previous_data_element_x 
        error_y_pos = current_data_element_y - previous_data_element_y
        x_pos_error_list = x_pos_error_list + [error_x_pos]
        y_pos_error_list = y_pos_error_list + [error_y_pos] 
    return [x_pos_error_list,y_pos_error_list]

#Read the provided files and load the calculated errors into to appropriate lists as follows
angle_error_list_1 = readAngleFilesAndCollectErrors("angles_1.csv")
x_pos_error_list_1,y_pos_error_list_1 = readXYCoordsFilesAndCollectErrors("xyCoords_1.csv")

#merge both angle_error_list_1 and angle_error_list_2 to a single list and make it a numpy array called 'angle_error_array'
angle_error_array = np.array(angle_error_list_1)
#merge both x_pos_error_list_1 and x_pos_error_list_2 to a single list and make it a numpy array called 'x_coord_error_array'
x_coord_error_array = np.array(x_pos_error_list_1)
#merge both y_pos_error_list_1 and y_pos_error_list_2 to a single list and make it a numpy array called 'y_coord_error_array'
y_coord_error_array = np.array(y_pos_error_list_1)

# Here we use 80% of the collected data  as the training set and 20% of the collected data as test set.
#TODO: Assign different propotions of the collected data set and test set and check how the test set error varies of the 
#Neural Network
data =  np.vstack((x_coord_error_array,y_coord_error_array)).T
target = np.vstack(angle_error_array)
data_input_tensor = torch.tensor(data.tolist()).float()
data_target_tensor = torch.tensor(target.tolist()).float()
data_with_target = torch.cat((data_input_tensor,data_target_tensor),1)
#TODO: what is the importance of using DataLoader utility function here?
loader= torch.utils.data.DataLoader(data_with_target,
                                     batch_size=data_with_target.size()[0], shuffle=True,
                                     num_workers=0)
#training set
train_set = []
#test set
test_set = []
for i in iter(loader):
    train_set_index = (int)(np.round(i.shape[0]*0.8))
    train_set = i[:train_set_index,:]
    test_set = i[train_set_index:,:]

print(train_set.shape)
print(test_set.shape)

#Defining Neural Network Model
class NN(torch.nn.Module):
    def __init__(self,n_feature,n_hidden1,n_hidden2,n_output):
        super(NN,self).__init__()
        self.hidden1 = torch.nn.Linear(n_feature,n_hidden1)
        self.do1 = torch.nn.Dropout(0.15)
        #self.relu1 = torch.nn.LeakyReLU()
        #self.bn1 = torch.nn.BatchNorm1d(n_hidden1,affine=False)
        self.hidden2 = torch.nn.Linear(n_hidden1,n_hidden2)
        #self.bn2 = torch.nn.BatchNorm1d(n_hidden2,affine=False)
        #self.relu2 = torch.nn.LeakyReLU()
        self.do2 = torch.nn.Dropout(0.1)
        self.predict = torch.nn.Linear(n_hidden2,n_output)
        
        
    def forward(self,x):
        x = self.hidden1(x)
        x = torch.sigmoid(x)
        #x = self.do1(x)
        x = self.hidden2(x)
        x = torch.sigmoid(x)
        #x = self.do2(x)
        x = self.predict(x)
        return x


train_set_inputs = train_set[:,:2]
#TODO: calculate the mean value of the train_set_inputs.
mean_of_train_input = torch.mean(train_set_inputs,0)
#standard deviation of the train set inputs.
std_of_the_train_input = torch.std(train_set_inputs,0)
#here we normalize the inputs of the neural network. What is the importance of that?
normalized_train_set_inputs = (train_set_inputs - mean_of_train_input)/std_of_the_train_input
#targets of the training set
train_set_targets = train_set[:,2][:,np.newaxis]
print(normalized_train_set_inputs.shape)
print(train_set_targets.shape)


#reload the your best neural network model with saved parameters
n_hidden1 = torch.load('best_nn_hidden1.pth')
n_hidden2 = torch.load('best_nn_hidden2.pth')
NN_model = NN(n_feature=2,n_hidden1=n_hidden1,n_hidden2=n_hidden2, n_output=1)
NN_model.load_state_dict(torch.load('best_nn_model.pth'))
#TODO: Extract inputs of the test_set
test_set_inputs = test_set[:,:2]
#TODO: Extract test set targets from the test_set
test_set_targets = test_set[:,2][:,np.newaxis]
#TODO: Normalize test set inputs by using the mean and standard deviation of the inputs of the training set
mean_training_inputs = torch.load('best_mean.pth')
std_training_inputs = torch.load('best_std.pth')
normalized_test_set_inputs = (test_set_inputs - mean_training_inputs)/std_training_inputs
#TODO: feed the normalized test set inputs to the Neural Network model and obtain the prediction for the test set.
prediction_test = NN_model(normalized_test_set_inputs)
print(prediction_test.shape)

# Grab an image and locate the largest green object:
def getPos():
    cam = ct.prepare_camera()
    while True:
        img = ct.capture_image(cam)
        x, y = ct.locate(img)
        if x is not None:
            break
    # print("Now the camera is done adjusting!")
    X,Y = [],[] 
    for _ in range(10):
        img = ct.capture_image(cam)
        x, y = ct.locate(img)
        # print(x, y)
        X.append(x)
        Y.append(y)
    cam.release()
    X = np.rint(np.mean(np.asarray(X))).astype(int)
    Y = np.rint(np.mean(np.asarray(Y))).astype(int)
    return (X,Y)


def squaredist(target_x,target_y,x,y):
    a = np.square(np.abs(target_x-x))
    b = np.square(np.abs(target_y-y))
    return np.sqrt(a+b)

PORT: /dev/ttyACM1, NAME: Fable Dongle 1.5, HWID: USB VID:PID=03EB:FABE LOCATION=1-2:1.0
Trying to connect /dev/ttyACM1 Linux implementation - tested!
dongle connect: True
Dongle...success
setup done
Runtime Created BE7 
Module IDs:  ['BE7 ']
Battery level: 100 %


QObject::moveToThread: Current thread (0x55b5e7093bb0) is not the object's thread (0x55b5e731c910).
Cannot move to target thread (0x55b5e7093bb0)

QObject::moveToThread: Current thread (0x55b5e7093bb0) is not the object's thread (0x55b5e731c910).
Cannot move to target thread (0x55b5e7093bb0)

QObject::moveToThread: Current thread (0x55b5e7093bb0) is not the object's thread (0x55b5e731c910).
Cannot move to target thread (0x55b5e7093bb0)

QObject::moveToThread: Current thread (0x55b5e7093bb0) is not the object's thread (0x55b5e731c910).
Cannot move to target thread (0x55b5e7093bb0)

QObject::moveToThread: Current thread (0x55b5e7093bb0) is not the object's thread (0x55b5e731c910).
Cannot move to target thread (0x55b5e7093bb0)

QObject::moveToThread: Current thread (0x55b5e7093bb0) is not the object's thread (0x55b5e731c910).
Cannot move to target thread (0x55b5e7093bb0)

QObject::moveToThread: Current thread (0x55b5e7093bb0) is not the object's thread (0x55b5e731c910).
Cannot move to tar

[0 0 0]
[255 113 255]


QObject::moveToThread: Current thread (0x55b5e7093bb0) is not the object's thread (0x55b5e731c910).
Cannot move to target thread (0x55b5e7093bb0)

QObject::moveToThread: Current thread (0x55b5e7093bb0) is not the object's thread (0x55b5e731c910).
Cannot move to target thread (0x55b5e7093bb0)

QObject::moveToThread: Current thread (0x55b5e7093bb0) is not the object's thread (0x55b5e731c910).
Cannot move to target thread (0x55b5e7093bb0)

QObject::moveToThread: Current thread (0x55b5e7093bb0) is not the object's thread (0x55b5e731c910).
Cannot move to target thread (0x55b5e7093bb0)

QObject::moveToThread: Current thread (0x55b5e7093bb0) is not the object's thread (0x55b5e731c910).
Cannot move to target thread (0x55b5e7093bb0)

QObject::moveToThread: Current thread (0x55b5e7093bb0) is not the object's thread (0x55b5e731c910).
Cannot move to target thread (0x55b5e7093bb0)

QObject::moveToThread: Current thread (0x55b5e7093bb0) is not the object's thread (0x55b5e731c910).
Cannot move to tar

torch.Size([528, 3])
torch.Size([132, 3])
torch.Size([528, 2])
torch.Size([528, 1])
torch.Size([132, 1])


In [2]:
# Initialize CMAC
from cmac2 import CMAC
# n_rfs = 3
# xmin = [172,172]
# xmax = [485,485]
# cmac = CMAC(n_rfs, xmin, xmax, 0.1)

err_treshold = 20
def ControlLoopWithBothNNandCMAC(target__x_coordinate,target__y_coordinate):
    number_of_iterations_for_convergence = 0
    
    # Initialize NN    
    NN_model = NN(n_feature=2,n_hidden1=n_hidden1,n_hidden2=n_hidden2, n_output=1)
    NN_model.load_state_dict(torch.load('best_nn_model.pth'))
    mean_training_inputs = torch.load('best_mean.pth')
    std_training_inputs = torch.load('best_std.pth')
    # Get x and y coodinates of the green box placed on the end effector of the robot
    robot_current_X_pos, robot_current_Y_pos = getPos()
    
    # Here we loop for 50 iterations assuming that 
    # the controller should achieve the desired target within atmost 50 iterations

    x_error_list,y_error_list, sqrt_error_list = [],[],[]
    for i in range(20):
        print("Curr_pos: ",robot_current_X_pos, robot_current_Y_pos)
        print("Curr_target:",target__x_coordinate,target__y_coordinate )
        
        x_coord_error = target__x_coordinate - robot_current_X_pos
        y_coord_error = target__y_coordinate - robot_current_Y_pos
        print("Error_x: ",x_coord_error)
        if (np.abs(x_coord_error) < err_treshold and np.abs(y_coord_error) < err_treshold):
            print("Number of iterations for convergence = ", number_of_iterations_for_convergence)
            x_error_list.append(np.abs(x_coord_error))
            y_error_list.append(np.abs(y_coord_error))
            sqrt_error_list.append(squaredist(target__x_coordinate,target__y_coordinate,robot_current_X_pos,robot_current_Y_pos))
            break
        xy_input_nn_model = torch.tensor([x_coord_error,y_coord_error]).float()
        
        # Normalize the input to the Neural network model using meaning and variance of the training set inputs.
        normalize_xy_input_nn_model = (xy_input_nn_model - mean_training_inputs)/std_training_inputs
        deltaY_nn = NN_model(normalize_xy_input_nn_model)
        print("DeltaY_nn: ",deltaY_nn.detach().numpy()[0])
        
        # Predict and train CMAC 
        deltaY_cmac = cmac.predict([target__x_coordinate, robot_current_X_pos])
        # deltaY_cmac = cmac.predict([robot_current_Y_pos, target__y_coordinate])
        cmac.learn(deltaY_nn.detach().numpy()[0])
        print("DeltaY_cmac: ",deltaY_cmac)
        
        # Get current motorY angle
        currAngY = api.getPos('Y',moduleID)
        print("Curr_angleY: ",currAngY)
        
        # Set new angleY
        robot_next_Y_pos = deltaY_nn + deltaY_cmac + currAngY
        api.setPos(-90,robot_next_Y_pos, moduleID)
        api.sleep(1.5)
        print("AngleY after: ",api.getPos('Y',moduleID))
        
        # Get current position of the robot in the camera frame
        robot_current_X_pos, robot_current_Y_pos = getPos()
        number_of_iterations_for_convergence = number_of_iterations_for_convergence + 1
        x_error_list.append(np.abs(x_coord_error))
        y_error_list.append(np.abs(y_coord_error))
        sqrt_error_list.append(squaredist(target__x_coordinate,target__y_coordinate,robot_current_X_pos,robot_current_Y_pos))
        print("\n")
        
    return number_of_iterations_for_convergence,x_error_list,y_error_list,sqrt_error_list

In [ ]:
# SAVE MODEL
import pickle
with open('cmac_model_test2.pkl', 'wb') as outp:
    pickle.dump(cmac, outp, pickle.HIGHEST_PROTOCOL)

In [3]:
# IMPORT MODEL
import pickle
with open('cmac_model_test2.pkl', 'rb') as inp:
    cmac = pickle.load(inp)

In [4]:
# IMPORT TARGET POINTS
targetList = torch.load('targetList_cmac1.pth')
targetList

[(424, 304),
 (394, 307),
 (366, 309),
 (384, 307),
 (396, 306),
 (210, 301),
 (304, 308),
 (208, 298),
 (361, 309),
 (322, 309)]

In [ ]:
# GENERATE RANDOM TARGET POINT
targetList = []
for i in range(10):
    if i < 5:
        angY = np.random.randint(-90,40)
        api.setPos(-90,angY,moduleID)
        api.sleep(1.5)
        targetList.append(getPos())
    else:
        angY = np.random.randint(-40,90)
        api.setPos(-90,angY,moduleID)
        api.sleep(1.5)
        targetList.append(getPos())
torch.save(targetList, 'targetList_cmac1.pth')

In [5]:
# LOOP THROUGH ALL TARGET POINTS
itList = []
eeList = []
x_error_list, y_error_list, sqrt_error_list = [],[],[]
i=0
err_treshold=10
for el in targetList:
    if i < 5 :
        api.setPos(-90,90,moduleID)
        api.sleep(1.5)
        i+=1
    else:
        api.setPos(-90,-90,moduleID)
        api.sleep(1.5)
        i+=1
    eeList.append([api.getPos('X',moduleID),api.getPos('Y',moduleID)])
    it, x_error, y_error, sqrt_error =  ControlLoopWithBothNNandCMAC(el[0],el[1])
    x_error_list.append(x_error[-1])
    y_error_list.append(y_error)
    sqrt_error_list.append(sqrt_error)
    itList.append(it)

Curr_pos:  208 297
Curr_target: 424 304
Error_x:  216
DeltaY_nn:  -21.17982
DeltaY_cmac:  -57.8783533038786
Curr_angleY:  85.25390625
AngleY after:  8.49609375


Curr_pos:  300 308
Curr_target: 424 304
Error_x:  124
DeltaY_nn:  -20.55755
DeltaY_cmac:  -27.460740636799024
Curr_angleY:  8.49609375
AngleY after:  -36.62109375


Curr_pos:  410 304
Curr_target: 424 304
Error_x:  14
DeltaY_nn:  -3.0660267
DeltaY_cmac:  33.28688056016431
Curr_angleY:  -36.62109375
AngleY after:  -10.25390625


Curr_pos:  352 308
Curr_target: 424 304
Error_x:  72
DeltaY_nn:  -20.67227
DeltaY_cmac:  -0.8771923196369116
Curr_angleY:  -10.25390625
AngleY after:  -29.58984375


Curr_pos:  396 306
Curr_target: 424 304
Error_x:  28
DeltaY_nn:  -19.181581
DeltaY_cmac:  21.90891488367624
Curr_angleY:  -29.58984375
AngleY after:  -29.00390625


Curr_pos:  395 306
Curr_target: 424 304
Error_x:  29
DeltaY_nn:  -19.238146
DeltaY_cmac:  18.125414731867142
Curr_angleY:  -29.00390625
AngleY after:  -29.58984375


Curr_pos:  

KeyboardInterrupt: 

In [ ]:
yhat = []
for el in eeList:
    if el[1] > 0:
        yhat.append(0)
    else:
        yhat.append(1)

%matplotlib qt 
fig = plt.figure()
ax = plt.axes()
x1 = linspace(0,len(targetList),len(targetList))
ax.set_ylim([0, 20+1])
plt.title("Iterations with CMAC, after training, threshold: {}".format(err_treshold))
plt.ylabel("Number of iterations")
for i in range(len(targetList)):
    # label = "MotorAngle: {:.2f}°, {:.2f}°\nTargetPos: {:.2f}, {:.2f}".format(eeList[i][0],eeList[i][1],targetList[i][0],targetList[0][1])
    label = "{:.2f}° / {:.2f}°\n{:.2f} / {:.2f}".format(eeList[i][0],eeList[i][1],targetList[i][0],targetList[0][1])
    plt.annotate(label, # this is the text
                 (x1[i],itList[i]), # these are the coordinates to position the label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center') # horizontal alig
scatter1= ax.scatter(x1[:5],itList[:5], c = 'blue',label='End-Effector in (-90°,90°)')
scatter2 = ax.scatter(x1[5:],itList[5:], c = 'red',label='End-Effector in (-90°,-90°)')
legend = ax.legend(loc='upper left')
ax.add_artist(legend)

## PLOT ERROR ON X
ax2=ax.twinx()
ax2.scatter(x1,x_error_list, c = 'green', label='X-pixel error')
ax2.set_ylim([-0.50, np.max(x_error_list)+1])
ax2.set_ylabel("Pixel",color="green")
legend = ax2.legend()
ax2.add_artist(legend)

plt.grid()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from numpy.core.function_base import linspace
fig,ax = plt.subplots()
x1 = linspace(0,10,10)
# ax.set_ylim([-0.50, np.max(x_er1ror_list)])
plt.title("Iterations with CMAC, threshold: {}, no lights".format(10))
ax.set_xlabel("Target-i")
ax.set_ylabel("Number of iterations")
scatter1 = ax.scatter(x1[:5],x1[:5], c = 'blue',label='End-Effector in (-90°,90°)')
scatter2  = ax.scatter(x1[5:],x1[5:], c = 'red',label='End-Effector in (-90°,90°)')
legend = ax.legend()
ax.add_artist(legend)
plt.grid()

plt.show()